Start with:
Data Check
Data cleansing
Verification that everything is proper

Analysis to do:
Country Analysis - Which country uses the most. BY $ amount and quantity
Product  Analytics  - Top products per country  
Price Analytics - By Month/Country
What sells the most, What is returned the most
Monthly ratio of successful sold to cancelled
Distribution between cancelled and successful invoice
Analysis of invoice by hourly distribution
Who are our top customers? By unit price, by quantity etc. Understand their needs and if data is skewed because of them eg. Big companies making big orders on a given product. (Might not mean it is favored by everyone). Basically check uniqueness in customer for given product.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import random
import os
import glob

%matplotlib inline
# Make plots larger
plt.rcParams['figure.figsize'] = (15, 9)

In [2]:
ds = pd.read_excel("data_science_analytics_2018_data.xlsx",dtype=str)
#Save Orignal Data

In [3]:
#convert data into proper format
ds['Quantity'] = pd.to_numeric(ds['Quantity'])
ds['UnitPrice'] = pd.to_numeric(ds['UnitPrice'])
ds['InvoiceDate'] = pd.to_datetime(ds['InvoiceDate'])

In [4]:
ds.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,C581569,20979,36 PENCILS TUBE RED RETROSPOT,-5,2011-12-09 11:58:00,1.25,17315,United Kingdom
1,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315,United Kingdom
2,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311,United Kingdom
3,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498,United Kingdom
4,C581490,22178,VICTORIAN GLASS HANGING T-LIGHT,-12,2011-12-09 09:57:00,1.95,14397,United Kingdom


In [5]:
cpy =ds

In [6]:
ds.describe()

,Quantity,UnitPrice
count,541909.000000,541909.000000
mean,9.552250,4.611114
std,218.081158,96.759853
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


In [7]:
ds.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID             object
Country                object
dtype: object

In [8]:
ds.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

ds.dropna(axis =0, subset=['CustomerID'],inplace=True)

In [9]:
ds.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [10]:
ds_top=ds[ds['InvoiceNo'].str.contains("C|c")==False]
ds_top.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
9288,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,nan,United Kingdom
9289,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,nan,United Kingdom
9290,A563185,B,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,nan,United Kingdom
9291,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680,France
9292,581587,22367,CHILDRENS APRON SPACEBOY DESIGN,8,2011-12-09 12:50:00,1.95,12680,France


In [11]:
ds['InvoiceNo'].dtypes

dtype('O')

In [12]:
ds['StockCode'].value_counts().head()

85123A    2313
22423     2203
85099B    2159
47566     1727
20725     1639
Name: StockCode, dtype: int64

In [13]:
ds_top['StockCode_total'] = ds_top['Quantity'] * ds_top['UnitPrice']

C:\Users\Rahil\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
ds.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,C581569,20979,36 PENCILS TUBE RED RETROSPOT,-5,2011-12-09 11:58:00,1.25,17315,United Kingdom
1,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315,United Kingdom
2,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311,United Kingdom
3,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498,United Kingdom
4,C581490,22178,VICTORIAN GLASS HANGING T-LIGHT,-12,2011-12-09 09:57:00,1.95,14397,United Kingdom


In [15]:
ds_top.groupby(['StockCode'])[["StockCode_total"]].sum().head(10)

,StockCode_total
StockCode,
10002,759.89
10080,119.09
10120,40.53
10123C,3.25
10123G,0.00
10124A,6.72
10124G,7.14
10125,994.84
10133,1544.22


In [16]:
ref = ds_top.sort_values(['StockCode_total'],ascending=[0])

In [17]:
ref

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,StockCode_total
10771,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446,United Kingdom,168469.60
481328,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346,United Kingdom,77183.60
323242,556444,22502,PICNIC BASKET WICKER 60 PIECES,60,2011-06-10 15:28:00,649.50,15098,United Kingdom,38970.00
527069,537632,AMAZONFEE,AMAZON FEE,1,2010-12-07 15:08:00,13541.33,nan,United Kingdom,13541.33
9290,A563185,B,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,nan,United Kingdom,11062.06
371728,551697,POST,POSTAGE,1,2011-05-03 13:46:00,8142.75,16029,United Kingdom,8142.75
199841,567423,23243,SET OF TEA COFFEE SUGAR TINS PANTRY,1412,2011-09-20 11:05:00,5.06,17450,United Kingdom,7144.72
490156,540815,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,3114,2011-01-11 12:55:00,2.10,15749,United Kingdom,6539.40
384332,550461,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,3114,2011-04-18 13:20:00,2.10,15749,United Kingdom,6539.40
128077,573003,23084,RABBIT NIGHT LIGHT,2400,2011-10-27 12:11:00,2.08,14646,Netherlands,4992.00
